# three-tier-geolocation - REST API Server

In [1]:
import requests
import os
# Declare constants
# REST_URL = "http://127.0.0.1:5000/uploader"

REST_URL = "http://127.0.0.1:5000"

DATASETS_PATH = "..//Datasets"

In [2]:
REQUESTED_FILE_NAME = "results.txt" 

def use_api(dataset_1p, dataset_3p, rtt_method, geolocation_method):
    output_file = os.path.join(OUTPUT_PATH, f"results_learn={dataset_1p}_test={dataset_3p}.txt")
    
    print("start:", dataset_1p, dataset_3p)
    
    dataset_1p_path = os.path.join(DATASETS_PATH, dataset_1p)
    dataset_3p_path = os.path.join(DATASETS_PATH, dataset_3p)
    
    FILE_MEASUREMENTS_1P = os.path.join(dataset_1p_path, "measurements-1party.csv")
    FILE_MEASUREMENTS_3P = os.path.join(dataset_3p_path, "measurements-3party.csv")
    FILE_SERVERS_1P = os.path.join(dataset_1p_path, "servers-1party.csv")
    FILE_SERVERS_3P = os.path.join(dataset_3p_path, "servers-3party.csv")
    FILE_DATACENTERS = os.path.join(dataset_3p_path, "datacenters.csv")
    FILE_SOLUTION = os.path.join(dataset_3p_path, "solution.csv")

    response = requests.post(REST_URL + "/rest",
                             files=[
                                 ('measurements-1party', ('measurements-1party.csv', open(FILE_MEASUREMENTS_1P, 'rb'), 'application/octet-stream')),
                                 ('servers-1party', ('servers-1party.csv', open(FILE_SERVERS_1P, 'rb'), 'application/octet-stream')),
                                 ('measurements-3party', ('measurements-3party.csv', open(FILE_MEASUREMENTS_3P, 'rb'), 'application/octet-stream')),
                                 ('servers-3party', ('servers-3party.csv', open(FILE_SERVERS_3P, 'rb'), 'application/octet-stream')),
                                 ('datacenters', ('datacenters.csv', open(FILE_DATACENTERS, 'rb'), 'application/octet-stream')),
                                 ('solution', ('solution.csv', open(FILE_SOLUTION, 'rb'), 'application/octet-stream'))
                             ],
                             data={'rtt_method': rtt_method, 'geolocation_method': geolocation_method})

    if response.status_code != 200:
        print("Error: POST received bad response code:", response.status_code)
        return

    requested_file_url = response.json()['Assets'][REQUESTED_FILE_NAME]
    print(requested_file_url)

    second_response = requests.get(requested_file_url)

    if second_response.status_code != 200:
        print("Error: GET received bad response code:", response.status_code)
        return

    with open(output_file, 'w', encoding="utf-8") as f:
        f.writelines(second_response.text)
    return second_response.text

## Use the API

### Single request

In [38]:
rtt_method = "Optimization"  # rtt_method = "Subtraction", "Optimization"
geolocation_method = "Multilateration"  # geolocation_method = "Multilateration", "Profiling"
OUTPUT_PATH = "./api-client-out/temp"  # make sure this directory exists
use_api(dataset_1p="DS-F3", dataset_3p="DS-F4", rtt_method=rtt_method, geolocation_method=geolocation_method)


start: DS-F3 DS-F4
http://127.0.0.1:5000/GetFile/tmpfo8v2fou/output/results.txt


### Multiple requests 

In [11]:
rtt_method = "Optimization"  # rtt_method = "Subtraction", "Optimization"
geolocation_method = "Profiling"  # geolocation_method = "Multilateration", "Profiling"
OUTPUT_PATH = f"api-client-out/{rtt_method}_{geolocation_method}"  # make sure this directory exists

DATASETS = ["DS-B1", "DS-F1", "DS-F2", "DS-F3", "DS-F4", ]

for d1 in DATASETS:
    for d2 in DATASETS:
        use_api(dataset_1p=d1, dataset_3p=d2, rtt_method=rtt_method, geolocation_method=geolocation_method)

start: DS-B1 DS-B1
http://127.0.0.1:5000/GetFile/tmpujw_fdsm/output/results.txt
start: DS-B1 DS-F1


KeyboardInterrupt: 

### Use one DC in each continent

In [3]:
import csv
from itertools import product
import json

rtt_method = "Optimization"  # rtt_method = "Subtraction", "Optimization"
geolocation_method = "Multilateration"  # geolocation_method = "Multilateration", "Profiling"
OUTPUT_PATH = "./api-client-out/temp"  # make sure this directory exists

def text_to_stats(text):
    mean_error = 0
    max_error = 0
    RMSE = 0
    success = 0
    for line in text.splitlines():
        if "Final-Geolocation Mean Error:" in line:
            parts = line.split()
            mean_error = parts[-2]
        elif "Final-Geolocation Max Error:" in line:
            parts = line.split()
            max_error = parts[-2]

        elif "Final-Geolocation RMSE Error:" in line:
            parts = line.split()
            RMSE = parts[-2]

        elif "Success rate:" in line:
            parts = line.split()
            success  = parts[-2] 
    return mean_error, max_error, RMSE, success

def clear_directories(path):
    for filename in os.listdir(path):
        file_path = os.path.join(path, filename)
        os.remove(file_path)

def filter_first_party(combination, first_party):
    filtered_rows = []
    file_names = []
    first_party_servers = f"..//Datasets//{first_party}//servers-1party.csv"
    first_party_measurments = f"..//Datasets//{first_party}//measurements-1party.csv"
    with open(first_party_servers, mode='r') as file:
        reader = csv.reader(file)
        for row in reader: # filter the servers (only keep the append to disable)
            if len(row) > 2 and row[0] == "file" and row[2] not in combination:
                file_names.append(row[1])
                filtered_rows.append(row)
            else:
                filtered_rows.append(row)
    # make sure the dir exists
    with open('../Datasets/temp-first/servers-1party.csv', mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerows(filtered_rows)
    filtered_rows = []
    #filter the measurements (only keep the append to disable)
    with open(first_party_measurments, mode='r') as file:
        reader = csv.reader(file)
        for row in reader:
            # if row[2] not in file_names: 
            filtered_rows.append(row)
    with open('../Datasets/temp-first/measurements-1party.csv', mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerows(filtered_rows)     
    return file_names

def filter_third_parth(third_party, file_names, combination):
    filtered_rows = []
    third_party_servers = f"..//Datasets//{third_party}//servers-3party.csv"
    third_party_measurments = f"..//Datasets//{third_party}//measurements-3party.csv"
    third_party_DC = f"..//Datasets//{third_party}//datacenters.csv"
    third_party_solution = f"..//Datasets//{third_party}//solution.csv"

    with open(third_party_servers, mode='r') as file:
        reader = csv.reader(file)
        for row in reader: # filter the servers
            if len(row) > 1 and row[0] == "file" and row[1] in file_names:
                pass
            else:
                filtered_rows.append(row)
        with open('../Datasets/temp-third/servers-3party.csv', mode='w', newline='') as file:
            writer = csv.writer(file)
            writer.writerows(filtered_rows)
    filtered_rows = []
    with open(third_party_measurments, mode='r') as file:
        reader = csv.reader(file)
        for row in reader: # filter the measurments
            if row[2] not in file_names:
                filtered_rows.append(row)
        with open('../Datasets/temp-third/measurements-3party.csv', mode='w', newline='') as file:
            writer = csv.writer(file)
            writer.writerows(filtered_rows)
    filtered_rows = []
    with open(third_party_DC, mode='r') as file:
        reader = csv.reader(file)
        for row in reader: # add learn_only as needed
            if row[0] not in combination:
                row[-1] = "learn_only"
                row.append("")
            filtered_rows.append(row)
    with open('../Datasets/temp-third/datacenters.csv', mode='w', newline='') as file:
            writer = csv.writer(file)
            writer.writerows(filtered_rows)
    filtered_rows = []
    with open(third_party_solution,mode ='r') as file:
        reader = csv.reader(file)
        for row in reader: # filter the solution
            if row[0] not in file_names: 
                filtered_rows.append(row)
    with open('../Datasets/temp-third/solution.csv', mode='w', newline='') as file:
            writer = csv.writer(file)
            writer.writerows(filtered_rows)

def cycle_DC(dataset_1p="DS-F3", dataset_3p="DS-F4"):
    datacenters = {}
    with open(f'..//Datasets//{dataset_1p}//datacenters.csv', mode='r') as file:
        reader = csv.reader(file)
        for row in reader:
            name = row[0]
            continent = row[3]
            if continent not in datacenters:
                datacenters[continent] = []
            datacenters[continent].append(name)
    continents = list(datacenters.keys())
    combinations = list(product(*[datacenters[continent] for continent in continents]))
    results = {}
    for rtt_method in ["Subtraction", "Optimization"]:
        for geolocation_method in ["Multilateration","Profiling"]:
            total_combinations = 0
            total_mean_error = 0
            total_max_error = 0
            total_RMSE = 0
            total_success = [0,0]
            for combination in combinations:
                total_combinations += 1
                file_names = filter_first_party(combination, dataset_1p)
                filter_third_parth(dataset_3p, file_names, combination)
                text = use_api(dataset_1p="temp-first", dataset_3p="temp-third", rtt_method=rtt_method, geolocation_method=geolocation_method)
                mean_error, max_error, RMSE, success = text_to_stats(text)
                total_mean_error += float(mean_error)
                total_max_error += float(max_error)
                total_RMSE += float(RMSE)
                total_success[0] += int(success.split("/")[0])
                total_success[1] += int(success.split("/")[1])
                clear_directories("../Datasets/temp-first")
                clear_directories("../Datasets/temp-third")
            total_mean_error /= total_combinations
            total_max_error /= total_combinations
            total_RMSE /= total_combinations
            results.update({(rtt_method,geolocation_method):[total_mean_error, total_max_error, total_RMSE, total_success]})
    return results
results = cycle_DC()
print(results)

start: temp-first temp-third
http://127.0.0.1:5000/GetFile/tmp8la0ik6x/output/results.txt
start: temp-first temp-third
http://127.0.0.1:5000/GetFile/tmptv38l_7n/output/results.txt
start: temp-first temp-third
http://127.0.0.1:5000/GetFile/tmp8w8d3gpf/output/results.txt
start: temp-first temp-third
http://127.0.0.1:5000/GetFile/tmpp1ggaut4/output/results.txt
start: temp-first temp-third
http://127.0.0.1:5000/GetFile/tmpzy5h8nxc/output/results.txt
start: temp-first temp-third
http://127.0.0.1:5000/GetFile/tmp9zjxa856/output/results.txt
start: temp-first temp-third
http://127.0.0.1:5000/GetFile/tmp4rxiqfx7/output/results.txt
start: temp-first temp-third
http://127.0.0.1:5000/GetFile/tmpsyhmu_jf/output/results.txt
start: temp-first temp-third
http://127.0.0.1:5000/GetFile/tmpgeottsv2/output/results.txt
start: temp-first temp-third
http://127.0.0.1:5000/GetFile/tmpsgyiha5k/output/results.txt
start: temp-first temp-third
http://127.0.0.1:5000/GetFile/tmp258cx76l/output/results.txt
start: tem

TypeError: keys must be str, int, float, bool or None, not tuple